In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, PeftConfig

import torch

In [2]:
torch_dtype = torch_dtype = torch.float16

In [3]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)

In [4]:
peft_model_id = "C:/Users/Harvey/Downloads/model-save"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
# model = AutoModelForCausalLM.from_pretrained(
#     'nijalmot/llama2-ko-dementia-half',
#     device_map={"": 0},
#     trust_remote_code=True
# )
model.config.use_cache = False
model.config.pretraining_tp = 1

In [6]:
# tokenizer = AutoTokenizer.from_pretrained(
#               'nijalmot/llama2-ko-dementia-half',
#               trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [7]:
import time

start = time.time()
prompt = "알츠하이머병 진단 방법에 대해 알려주세요"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, truncation=True)
result = pipe(f"<s>[INST] {prompt} [/INST]")
now = time.time()
print(result[0]['generated_text'])
print(f"generate time : {now - start: .4f}")

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausa

c:\Users\Harvey\anaconda3\envs\ml_dl\lib\site-packages\transformers\models\llama\modeling_llama.py:602: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<s>[INST] 알츠하이머병 진단 방법에 대해 알려주세요 [/INST] 알츠하이머병은 치매의 가장 흔한 형태이며, 주로 노인들에게서 발생하는 퇴행성 뇌질환입니다. 이 질환은 기억력 저하와 인지기능 저하를 특징으로 하며, 서서히 진행됩니다.알츠하이머병은 주로 65세 이상의 노인들에게서 발생하며, 65세 이상의 노인 중 약 10%가 이 질환을 앓고 있습니다. 이 질환은 기억력 저하와 인지기능 저하를 특징으로 하며, 서서히 진행됩니다. 초기에는 최근 기억력 저하와 언어능력 저하가 나타나고, 진행됨에 따라 기억력 저하와 인지기능 저하가 심해집니다. 또한, 알츠하이머병은 진행됨에 따라 인지기능 저하와 함께 행동 변화와 우울증, 망상, 환각, 공격성 등의 정신행동증상이 나타날
generate time :  180.7670
